# FeatureExtraction Class Demo

FeatureExtraction is a class to run deep feature synthesis from Featuretools using different training windows. The output is a Pandas dataframe with the specified features based on the `EntitySet` and `target_entity`. The class also has methods to calculate the average rate of change and difference between similar features aggregated on different time periods. The class requires a Featuretools `EntitySet` and any relationships to be defined.

In [2]:
import pandas as pd 
import numpy as np
import featuretools as ft
import featuretools.variable_types as vtypes
from FeatureExtraction import FeatureExtraction

## Creating datasets
We will make an `EntitySet` based on a simple churn example of three tables:
* `members` is the parent table that contains `id`, `gender`, and `registration_date`
* `trans` is the transactions table containing `id`, `amt_paid`, `transaction_date`, and `membership_expire_date`
* `cutoff_times` provides the `cutoff_time` for each `instance_id` (equivalent to `id`) at which data in other tables can be used in creating features. The `label` is also included, although this is optional. Here, 1 indicates churn, and 0 indicates no churn.

In [3]:
members = [[1,'f','2016-06-18'],
           [2,'f','2016-06-19'],
           [3,'m','2016-06-20']]

members = pd.DataFrame(members, columns =['id','gender','registration_date'])
members['registration_date'] = members['registration_date'].astype('datetime64[ns]')

In [4]:
members

,id,gender,registration_date
0,1,f,2016-06-18
1,2,f,2016-06-19
2,3,m,2016-06-20


In [5]:
trans = [[1,30,2,'2017-01-01','2017-02-28'],
        [1,60,4,'2017-02-25','2017-04-03'],
        [1,90,6,'2017-03-15','2017-03-16'],
        [2,250,8,'2017-01-01','2017-02-28'],
        [2,50,10,'2017-02-01','2017-03-15'],
        [2,100,2,'2017-04-30','2017-05-20'],
        [3,1000,3,'2017-07-01','2017-08-01'],
        [3,500,4,'2017-08-01','2017-09-01'],
        [3,250,5,'2017-10-15','2017-11-15']]

trans = pd.DataFrame(trans, columns = ['id','amt_paid', 'qty',
                                       'transaction_date',
                                       'membership_expire_date'])
trans[['transaction_date', 'membership_expire_date']] = trans[['transaction_date', 'membership_expire_date']].astype('datetime64[ns]')

In [6]:
trans

,id,amt_paid,qty,transaction_date,membership_expire_date
0,1,30,2,2017-01-01,2017-02-28
1,1,60,4,2017-02-25,2017-04-03
2,1,90,6,2017-03-15,2017-03-16
3,2,250,8,2017-01-01,2017-02-28
4,2,50,10,2017-02-01,2017-03-15
5,2,100,2,2017-04-30,2017-05-20
6,3,1000,3,2017-07-01,2017-08-01
7,3,500,4,2017-08-01,2017-09-01
8,3,250,5,2017-10-15,2017-11-15


In [7]:
cutoff_times = [[1, '2017-03-15', 0],
       [2, '2017-02-01', 1],
       [3, '2017-09-01', 1]]

cutoff_times = pd.DataFrame(cutoff_times, columns = ['instance_id',
                            'cutoff_time','label'])
cutoff_times['cutoff_time'] = cutoff_times['cutoff_time'].astype('datetime64[ns]')


In [8]:
cutoff_times

,instance_id,cutoff_time,label
0,1,2017-03-15,0
1,2,2017-02-01,1
2,3,2017-09-01,1


## Creating Featuretools Entities

In [9]:
# Make empty entityset
es = ft.EntitySet(id = 'customers')
# Create entity from members
es.entity_from_dataframe(entity_id='members', dataframe=members,
                         index = 'id', time_index = 'registration_date', 
                         variable_types = {'gender': vtypes.Categorical})


# Create entity from transactions
es.entity_from_dataframe(entity_id='transactions', dataframe=trans,
                         index = 'transactions_index', make_index = True,
                         time_index = 'transaction_date')
r_member_transactions = ft.Relationship(es['members']['id'], es['transactions']['id'])
es.add_relationships([r_member_transactions])
es.add_last_time_indexes()

## FeatureExtraction without Training Windows and Cutoff Times

To run Deep Feature Synthesis on an `EntitySet` without using cutoff times, we only need to specify the `target_entity`. For this example we used `sum` and `week` for the aggregate and transform functions respectively.

In [10]:
dfs = FeatureExtraction(es)
dfs.add_agg_primitives(['sum'])
dfs.add_trans_primitives(['week'])
dfs.dfsWindow('members')

The aggregate primitives have been added:  sum
The transformative primitives have been added:  week
Built 4 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████████████████████| Calculated: 1/1 chunks
---> 0 columns dropped due to duplicate values:



,gender,SUM(transactions.amt_paid),SUM(transactions.qty),WEEK(registration_date)
id,,,,
1,f,180,12,24
2,f,400,20,24
3,m,1750,12,25


## FeatureExtraction with Training Windows and Cutoff Times

Now we test the FeatureExtraction class on the `EntitySet` we created using a `training_window` of 1 and 2 months. A `training_window` of 1 month indicates features built within one month of the cutoff date, and 2 months indicates features built within two months of the cutoff date.

In [11]:
monthly = FeatureExtraction(es)
monthly.add_agg_primitives(['sum', 'mean'])
monthly.add_trans_primitives(['month'])
training_window = [1,2]
monthly.dfsWindow('members', 'monthly', training_window, cutoff_times)

The aggregate primitives have been added:  sum mean
The transformative primitives have been added:  month
Built 6 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|████████████████████████████████████████████| Calculated: 1/1 chunks
---> 6 columns dropped due to duplicate values:



,,gender,SUM(transactions.amt_paid),SUM(transactions.qty),MEAN(transactions.amt_paid),MEAN(transactions.qty),MONTH(registration_date),label,SUM(transactions.amt_paid)_1mos,SUM(transactions.qty)_1mos,MEAN(transactions.amt_paid)_1mos,MEAN(transactions.qty)_1mos,SUM(transactions.amt_paid)_2mos,SUM(transactions.qty)_2mos,MEAN(transactions.amt_paid)_2mos,MEAN(transactions.qty)_2mos
id,time,,,,,,,,,,,,,,,
2,2017-02-01,f,300,18,150,9.0,6,1,50.0,10.0,50.0,10.0,300,18,150,9
1,2017-03-15,f,180,12,60,4.0,6,0,150.0,10.0,75.0,5.0,150,10,75,5
3,2017-09-01,m,1500,7,750,3.5,6,1,0.0,0.0,NaN,NaN,500,4,500,4


In [12]:
monthly.feature_defs

['gender',
 'SUM(transactions.amt_paid)',
 'SUM(transactions.qty)',
 'MEAN(transactions.amt_paid)',
 'MEAN(transactions.qty)',
 'MONTH(registration_date)',
 'label',
 'SUM(transactions.amt_paid)_1mos',
 'SUM(transactions.qty)_1mos',
 'MEAN(transactions.amt_paid)_1mos',
 'MEAN(transactions.qty)_1mos',
 'SUM(transactions.amt_paid)_2mos',
 'SUM(transactions.qty)_2mos',
 'MEAN(transactions.amt_paid)_2mos',
 'MEAN(transactions.qty)_2mos']

To see that the features calculated are consistent with the original data, we can look at `SUM(transactions.amt_paid)` for id=1 and compare this to the original transactions table and cutoff times.

In [13]:
monthly.df.loc[1][['SUM(transactions.amt_paid)','SUM(transactions.amt_paid)_1mos','SUM(transactions.amt_paid)_2mos']]

,SUM(transactions.amt_paid),SUM(transactions.amt_paid)_1mos,SUM(transactions.amt_paid)_2mos
time,,,
2017-03-15,180,150.0,150


In [14]:
print(cutoff_times[cutoff_times['instance_id']==1])
trans[(trans['id'] == 1)][['amt_paid', 'transaction_date']]

   instance_id cutoff_time  label
0            1  2017-03-15      0


,amt_paid,transaction_date
0,30,2017-01-01
1,60,2017-02-25
2,90,2017-03-15


As shown above, the total amt_paid for id=1 from the cutoff date of 2017-03-15 is 90+60+30 = 180, which is consistent with `monthly.df`'s `SUM(transactions.amt_paid)`. Similarly, the sum of amt_paid within a month of the cutoff date includes amt_paid from 2017-03-15 and 2017-02-25, or 150. The sum of amt_paid within 2 months of the cutoff date is the same, considering that the next `transaction_date` before 2017-02-25 is more than two months from the `cutoff_time` of 2017-03-15. These values are also consistent with `monthly.df`'s `SUM(transactions.amt_paid)_1mos` and `SUM(transactions.amt_paid)_2mos` respectively.

### Calculating Average Rate of Change
Now we will run `calc_avg_change` on our `EntitySet`. This returns the average rate of change, or trend, between the start and end dates given by the user. For example, `calc_avg_change(1,2,'monthly')` will return the average rate of change between features that were based on 1 month and 2 months from the cutoff time [ie. (numeric_feature_2mos - numeric_feature_1mos) / (2-1) ]. These columns are suffixed `_1-2mos_avgchange`

In [15]:
monthly.calc_avg_change(1,2,'monthly')

,,gender,SUM(transactions.amt_paid),SUM(transactions.qty),MEAN(transactions.amt_paid),MEAN(transactions.qty),MONTH(registration_date),label,SUM(transactions.amt_paid)_1mos,SUM(transactions.qty)_1mos,MEAN(transactions.amt_paid)_1mos,MEAN(transactions.qty)_1mos,SUM(transactions.amt_paid)_2mos,SUM(transactions.qty)_2mos,MEAN(transactions.amt_paid)_2mos,MEAN(transactions.qty)_2mos,MEAN(transactions.amt_paid)_1-2mos_avgchange,MEAN(transactions.qty)_1-2mos_avgchange,SUM(transactions.amt_paid)_1-2mos_avgchange,SUM(transactions.qty)_1-2mos_avgchange
id,time,,,,,,,,,,,,,,,,,,,
2,2017-02-01,f,300,18,150,9.0,6,1,50.0,10.0,50.0,10.0,300,18,150,9,100.0,-1.0,250.0,8.0
1,2017-03-15,f,180,12,60,4.0,6,0,150.0,10.0,75.0,5.0,150,10,75,5,0.0,0.0,0.0,0.0
3,2017-09-01,m,1500,7,750,3.5,6,1,0.0,0.0,NaN,NaN,500,4,500,4,NaN,NaN,500.0,4.0


In [16]:
monthly.feature_defs

['gender',
 'SUM(transactions.amt_paid)',
 'SUM(transactions.qty)',
 'MEAN(transactions.amt_paid)',
 'MEAN(transactions.qty)',
 'MONTH(registration_date)',
 'label',
 'SUM(transactions.amt_paid)_1mos',
 'SUM(transactions.qty)_1mos',
 'MEAN(transactions.amt_paid)_1mos',
 'MEAN(transactions.qty)_1mos',
 'SUM(transactions.amt_paid)_2mos',
 'SUM(transactions.qty)_2mos',
 'MEAN(transactions.amt_paid)_2mos',
 'MEAN(transactions.qty)_2mos',
 'MEAN(transactions.amt_paid)_1-2mos_avgchange',
 'MEAN(transactions.qty)_1-2mos_avgchange',
 'SUM(transactions.amt_paid)_1-2mos_avgchange',
 'SUM(transactions.qty)_1-2mos_avgchange']

To check that the calculations work properly, we look at the `SUM(transactions.qty)` at 1 month and 2 months, and the average rate of change between the two. Based on the table below, the 1-2mos_avgchange column shows the correct difference between the sum at 2 months and at 1 month from cutoff date divided by their difference in months.

In [17]:
monthly.df[['SUM(transactions.qty)_1mos','SUM(transactions.qty)_2mos','SUM(transactions.qty)_1-2mos_avgchange']]

,,SUM(transactions.qty)_1mos,SUM(transactions.qty)_2mos,SUM(transactions.qty)_1-2mos_avgchange
id,time,,,
2,2017-02-01,10.0,18,8.0
1,2017-03-15,10.0,10,0.0
3,2017-09-01,0.0,4,4.0


### Calculating Difference
`calc_diff` is an alternative to `calc_avg_change` and only calculates the difference between two time periods specified by the user. Thus, `calc_diff(1,2,'monthly')` will return the difference between numeric features based on 2 months and 1 month from the cutoff date. These columns are suffixed `_1-2mos_diff`.

In [18]:
monthly.calc_diff(1,2,'monthly')

,,gender,SUM(transactions.amt_paid),SUM(transactions.qty),MEAN(transactions.amt_paid),MEAN(transactions.qty),MONTH(registration_date),label,SUM(transactions.amt_paid)_1mos,SUM(transactions.qty)_1mos,MEAN(transactions.amt_paid)_1mos,...,MEAN(transactions.amt_paid)_2mos,MEAN(transactions.qty)_2mos,MEAN(transactions.amt_paid)_1-2mos_avgchange,MEAN(transactions.qty)_1-2mos_avgchange,SUM(transactions.amt_paid)_1-2mos_avgchange,SUM(transactions.qty)_1-2mos_avgchange,MEAN(transactions.amt_paid)_1-2mos_diff,MEAN(transactions.qty)_1-2mos_diff,SUM(transactions.amt_paid)_1-2mos_diff,SUM(transactions.qty)_1-2mos_diff
id,time,,,,,,,,,,,,,,,,,,,,,
2,2017-02-01,f,300,18,150,9.0,6,1,50.0,10.0,50.0,...,150,9,100.0,-1.0,250.0,8.0,100.0,-1.0,250.0,8.0
1,2017-03-15,f,180,12,60,4.0,6,0,150.0,10.0,75.0,...,75,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-09-01,m,1500,7,750,3.5,6,1,0.0,0.0,NaN,...,500,4,NaN,NaN,500.0,4.0,NaN,NaN,500.0,4.0


In [19]:
monthly.feature_defs

['gender',
 'SUM(transactions.amt_paid)',
 'SUM(transactions.qty)',
 'MEAN(transactions.amt_paid)',
 'MEAN(transactions.qty)',
 'MONTH(registration_date)',
 'label',
 'SUM(transactions.amt_paid)_1mos',
 'SUM(transactions.qty)_1mos',
 'MEAN(transactions.amt_paid)_1mos',
 'MEAN(transactions.qty)_1mos',
 'SUM(transactions.amt_paid)_2mos',
 'SUM(transactions.qty)_2mos',
 'MEAN(transactions.amt_paid)_2mos',
 'MEAN(transactions.qty)_2mos',
 'MEAN(transactions.amt_paid)_1-2mos_avgchange',
 'MEAN(transactions.qty)_1-2mos_avgchange',
 'SUM(transactions.amt_paid)_1-2mos_avgchange',
 'SUM(transactions.qty)_1-2mos_avgchange',
 'MEAN(transactions.amt_paid)_1-2mos_diff',
 'MEAN(transactions.qty)_1-2mos_diff',
 'SUM(transactions.amt_paid)_1-2mos_diff',
 'SUM(transactions.qty)_1-2mos_diff']

To check that the calculations work properly, we look at the `SUM(transactions.amt_paid)` at 1 month, 2 months, and the difference between the two. Based on the table below, the `SUM(transactions.amt_paid)_1-2mos_diff` column shows the correct difference between the sum at 2 months and at 1 month from the cutoff time.

In [20]:
monthly.df[['SUM(transactions.amt_paid)_1mos','SUM(transactions.amt_paid)_2mos','SUM(transactions.amt_paid)_1-2mos_diff']]

,,SUM(transactions.amt_paid)_1mos,SUM(transactions.amt_paid)_2mos,SUM(transactions.amt_paid)_1-2mos_diff
id,time,,,
2,2017-02-01,50.0,300,250.0
1,2017-03-15,150.0,150,0.0
3,2017-09-01,0.0,500,500.0


## Conclusion
This notebook serves to demonstrate how the class FeatureExtraction can be used. Other options for dfsWindow to explore are:
* time_scope = 'weekly' or 'daily' for weekly and daily training windows
* chunk_size = specifies how many rows to compute on at a time, similar to batch size processing
* n_jobs = enables parallel core processing if specified

To get a better understanding on an application of the full automated modelling pipeline, please look at the feature_engineering_KKBOX notebook where the class is applied to the music subscription KKBOX dataset.

For development purposes, a full test for the entire class is available under the file test_FeatureExtraction.py. The file can be run on the command line terminal using `pytest pathname\test_FeatureExtraction.py`